# Getting to know HCIPy

Docs: 
https://hcipy.readthedocs.io/en/latest/

What do I need?
- aperture
- coronagraph
- optical propagation


In [ ]:
from hcipy import *
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Setting up pupil and focal grid and propagator
pupil_grid = make_pupil_grid(dims=1024, diameter=1)
focal_grid = make_focal_grid(pupil_grid, 8, 32)    # 8 pixels per lambda/D, 32 lambda/D radius of total image
prop = FraunhoferPropagator(pupil_grid, focal_grid)   # this is without a coronagraph

## Apertures

### Circular aperture

Grid and aperture normalization is something I struggle with in the beginning. Grids have a size, and their default is `size=1`. Apertures have a normalization and their default is usually `normalization=False`. This means that as long as I have an aperture with diameter 1 (meter) on a grid of size one, I will have the entire aperture displayed. If my aperture is larger than 1 and *not* normalized, the grid will be sampled only on a bit of the aperture of size of the aperture - we display only one part of the aperture. However, if the aperture is normalized, this means that while it may have an arbitrary physical size, it is actually normlaized to 1 and the grid of size 1 will be sampled over the entire aperture, and we're not chopping random bits off.

In [ ]:
circ = circular_aperture(diameter=1)
circ = evaluate_supersampled(circ, pupil_grid, 8)

imshow_field(circ)
plt.show()

### Hexagonal aperture

In [ ]:
hexag = hexagonal_aperture(circum_diameter=1)
hexag = evaluate_supersampled(hexag, pupil_grid, 8)

imshow_field(hexag)
plt.show()

### Magellan aperture

In [ ]:
magellan = make_magellan_aperture(normalized=True)
magellan = evaluate_supersampled(magellan, pupil_grid, 8)

imshow_field(magellan)
plt.show()

### Luvoir A aperture (old)

In [ ]:
luvoir = make_luvoir_a_aperture(normalized=True)
luvoir = evaluate_supersampled(luvoir, pupil_grid, 4)    # this takes quite a while

imshow_field(luvoir)
plt.show()

### HiCAT apeture

This is the ATLAST aperture Lucie used in her PASTIS paper so I will be focusing on this, but without spiders.

In [ ]:
hicat = make_hicat_aperture(normalized=True, with_spiders=False)
hicat = evaluate_supersampled(hicat, pupil_grid, 2)

imshow_field(hicat)

## Coronagraph

### Perfect coronagraph

#### Circular aperture

In [ ]:
# Create coronagraph propagator on aperture
coro_perf = PerfectCoronagraph(circ)

# Create a wavefront from aperture
wf = Wavefront(circ)

# Apply coronagraph propagator of wavefront to Lyot plane
lyot_plane = coro_perf(wf)

# Display
imshow_field(np.log10(lyot_plane.intensity), vmin=-8, vmax=0)
plt.colorbar()
plt.show()

In [ ]:
# Add Lyot stop and display image
lyot_stop = Apodizer(circular_aperture(0.99)(pupil_grid))
# circular aperture size is determined by grid it's made on. here the pupil --> 0.99*pupil_grid

img = prop(lyot_stop(lyot_plane))
img_ref = prop(wf)

imshow_field(np.log10(img.intensity / img_ref.intensity.max()), vmin=-12)
plt.title("Circular aperture perfect coronagraph")
plt.colorbar()
plt.show()

In [ ]:
# Compare to propagation without coronagraph
imshow_field(np.log10(img_ref.intensity), vmin=-8, vmax=0)
plt.title("Circular aperture reference image")
plt.colorbar()
plt.show()

#### Luvoir aperture

In [ ]:
# Create coronagraph propagator on aperture
coro_perf = PerfectCoronagraph(luvoir)

# Create a wavefront from aperture
wf = Wavefront(luvoir)

# Apply coronagraph propagator of wavefront to Lyot plane
lyot_plane = coro_perf(wf)

# Display
imshow_field(np.log10(lyot_plane.intensity), vmin=-8, vmax=0)
plt.colorbar()
plt.show()

In [ ]:
# Add Lyot stop and display image
lyot_stop = Apodizer(circular_aperture(0.99)(pupil_grid))   # circular Lyot stop on Luvoir aperture... not great but works

img = prop(lyot_stop(lyot_plane))
img_ref = prop(wf)

imshow_field(np.log10(img.intensity / img_ref.intensity.max()), vmin=-12)
plt.title("Luvoir aperture perfect coronagraph")
plt.colorbar()
plt.show()

In [ ]:
# Compare to propagation without coronagraph
imshow_field(np.log10(img_ref.intensity), vmin=-8, vmax=0)
plt.title("Luvoir iaperture reference image")
plt.colorbar()
plt.show()

### HiCAT aperture

In [ ]:
# Create coronagraph propagator on aperture
coro_perf = PerfectCoronagraph(hicat)

# Create a wavefront from aperture
wf = Wavefront(hicat)

# Apply coronagraph propagator of wavefront to Lyot plane
lyot_plane = coro_perf(wf)

# Display
imshow_field(np.log10(lyot_plane.intensity), vmin=-8, vmax=0)
plt.colorbar()
plt.show()

In [ ]:
# Add Lyot stop and display image
lyot_stop = Apodizer(circular_aperture(0.99)(pupil_grid))

img = prop(lyot_stop(lyot_plane))
img_ref = prop(wf)

imshow_field(np.log10(img.intensity / img_ref.intensity.max()), vmin=-12)
plt.title("HiCAT aperture perfect coronagraph")
plt.colorbar()
plt.show()

In [ ]:
# Compare to propagation without coronagraph
imshow_field(np.log10(img_ref.intensity), vmin=-8, vmax=0)
plt.title("HiCAT iaperture reference image")
plt.colorbar()
plt.show()

### Lyot coronagraph

#### Circular aperture

In [ ]:
# Create a focal plane mask
fpm = 1 - circular_aperture(4)(focal_grid)
# here circular aperture size is on focal_grid, meanin in terms of lamnda/D

# Create Lyot coronagraph on pupil grid with FPM
coro_lyot = LyotCoronagraph(pupil_grid, fpm)

# Create a wavefront from aperture
wf = Wavefront(circ)

# Apply coronagraph propagator of wavefront to Lyot plane
lyot_plane = coro_lyot(wf)

# Display
imshow_field(np.log10(lyot_plane.intensity), vmin=-8, vmax=0)
plt.title("Circ Lyot plane")
plt.colorbar()

In [ ]:
# Add Lyot stop and display image
lyot_stop = Apodizer(circular_aperture(0.99)(pupil_grid))

img = prop(lyot_stop(lyot_plane))
img_ref = prop(wf)

imshow_field(np.log10(img.intensity / img_ref.intensity.max()), vmin=-12)
plt.title("Circular aperture Lyot coronagraph")
plt.colorbar()
plt.show()

## Generate ATLAST pupil

Check Luvoir API for this, should be easy since I don't need spiders or omitting certain segments.

Honestly, I just need the HiCAT pupil without the spiders, but it needs to return the segment coordinates.

In [ ]:
def get_atlast_aperture(normalized=False, with_segment_gaps=True, segment_transmissions=1):
    '''Make the ATLAST pupil mask.

    This function is a copy of make_hicat_aperture(), except that it also returns the segment positions.

    Parameters
    ----------
    normalized : boolean
        If this is True, the outer diameter will be scaled to 1. Otherwise, the
        diameter of the pupil will be 15.0 meters.
    with_segment_gaps : boolean
        Include the gaps between individual segments in the aperture.
    segment_transmissions : scalar or array_like
        The transmission for each of the segments. If this is a scalar, this transmission will
        be used for all segments.

    Returns
    -------
    Field generator
        The ATLAST aperture.
    CartesianGrid
        The segment positions.
    '''
    pupil_diameter = 15. # m
    segment_circum_diameter = 2 / np.sqrt(3) * pupil_diameter / 7
    num_rings = 3
    segment_gap = 1e-2

    if not with_segment_gaps:
        segment_gap = 0

    if normalized:
        segment_circum_diameter /= pupil_diameter
        segment_gap /= pupil_diameter
        pupil_diameter = 1.0

    segment_positions = make_hexagonal_grid(segment_circum_diameter / 2 * np.sqrt(3), num_rings)
    segment_positions = segment_positions.subset(lambda grid: ~(circular_aperture(segment_circum_diameter)(grid) > 0))

    hexagon = hexagonal_aperture(segment_circum_diameter - segment_gap)
    
    def segment(grid):
        return hexagon(grid.rotated(np.pi/2))

    segmented_aperture = make_segmented_aperture(segment, segment_positions, segment_transmissions)

    def func(grid):
        res = segmented_aperture(grid)

        return Field(res, grid)
    return func, segment_positions

In [ ]:
atlast, seg_positions = get_atlast_aperture(normalized=True)
atlast = evaluate_supersampled(atlast, pupil_grid, 2)

imshow_field(atlast)

In [ ]:
print("x positions: \n{}".format(seg_positions.x))
print("y positions: \n{}".format(seg_positions.y))

Now from the slightly modified function in the new m odule for ATLAST and HCIPy coronagraphs.

In [ ]:
import os
os.chdir('../../pastis/')
from config import CONFIG_INI
import util_pastis as util

In [ ]:
import atlast_imaging as atim

In [ ]:
seg_coords = atim.get_atlast_aperture(outDir='/Users/ilaginja/', normalized=True)

In [ ]:
seg_position = np.zeros((36, 2))

print(seg_coords.x.shape)
print(seg_coords.y.shape)
print(type(seg_coords.x))

In [ ]:
seg_position[3].shape

## Mini-segment

Figure out how to generate and propagate mini-segment, since that is needed for PASTIS. It's already there, but I will have to adjust how to deal with it.

In [ ]:
mini = hexagonal_aperture(circum_diameter=1, angle=np.pi/2)
mini = evaluate_supersampled(mini, pupil_grid, 4)

imshow_field(mini)
plt.show()

In [ ]:
wf = Wavefront(mini)
test = prop(wf)

In [ ]:
imshow_field(np.log10(test.intensity))

`test` is a Wavefron object which means it is not really a 2D array:

In [ ]:
print(test.intensity.shape)

You can access the 2D array easily though, which can be useful when needingn a cropped array:

In [ ]:
test2d = test.intensity.shaped
print(test2d.shape)